1. Problem Statement

2. Hypothesis Generation

3. Getting the system ready and loading the data

4. Understanding the data

5. Exploratory Data Analysis

 	i. Perform Univariate Analysis

 	ii. Perform Bivariate Analysis

6. Missing value and outlier treatment

7. Evaluation Metrics for classification problem

8. Model building: part 1 (Apply Deep Learning classification algorithm without step 9)

9. Feature engineering

10. Model building: part 2 (Apply Deep Learning classification algorithm with step 9)

11. Model deployment - Dash app on https://www.render.com